# Preparation of GWR - Data

## Libraries and Settings

In [ ]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

## Download GWR data

Source: https://public.madd.bfs.admin.ch

In [ ]:
# Download latest file for Switzerland
url = "https://public.madd.bfs.admin.ch/buildings_ch.geojson"
response = requests.get(url)

# Ensure response is valid
if response.status_code == 200:
    # Open file in write mode and write the response content
    with open('buildings_ch.geojson', 'wb') as file:
        file.write(response.content)
else:
    print(f"Failed to download file, status code: {response.status_code}")

## Import latest GWR Data

In [ ]:
# Load json file
with open('buildings_ch.geojson') as f:
    data = json.load(f)

# Flatten nested json data
df_orig = pd.json_normalize(data, record_path=['features'])
df_orig

# Remove prefix
df_orig.columns = df_orig.columns.str.replace('properties.', '')
df_orig.columns = df_orig.columns.str.replace('geometry.', '')

# Create copy
df = df_orig.copy()
df.head()

## Separate Swiss LV95 coordinates

In [ ]:
# Separate coordinates
df['x_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['x_coord']
df['y_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['y_coord']

# Remove column 'coordinates'
df = df.drop(['coordinates'], axis=1)
df

## Create WGS84 coordinates

In [ ]:
# Create geodataframe and calculate latitude and longitude
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x_coords'], df['y_coords']), crs="EPSG:2056")

# Convert the Swiss LV95 coordinates to lat & lon
gdf = gdf.to_crs(epsg=4326)

# Get Latitude and Longitude
df['latitude'] = gdf['geometry'].y
df['longitude'] = gdf['geometry'].x

# Show data
df

## Plot subset of buildings

In [ ]:
# Subset
df_sub = df.loc[df['municipalityName'] == 'Greifensee'].sample(100).dropna()
df_sub

# Create the map
m = folium.Map(location=[df_sub['latitude'].mean(), df_sub['longitude'].mean()], zoom_start=15)

# Add points to the map
for idx, row in df_sub.iterrows():
    folium.Marker(location=([row['latitude'], 
                            row['longitude']]),
                  popup=row['buildingCategory']).add_to(m)

# Display the map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')